# First verification, heat equation

Author: Jukka Aho

**Abstract**: ?

## Problem description


## Theory / solution

- Reference, uncertainty

## Implementation in JuliaFEM

- Mesh, boundary conditions, etc.

In [1]:
using JuliaFEM: Field, Quad4, Seg2, PlaneHeatProblem, DirichletProblem
using JuliaFEM: new_fieldset!, add_field!, add_element!, SimpleSolver, add_problem!, interpolate
using FactCheck

# Define Problem 1:
# - Field function: Laplace equation Δu=0 in Ω={u∈R²|(x,y)∈[0,1]×[0,1]}
# - Neumann boundary on Γ₁={0<=x<=1, y=0}, ∂u/∂n=600 on Γ₁

el1 = Quad4([1, 2, 3, 4])
new_fieldset!(el1, "geometry")
add_field!(el1, "geometry", Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0], [1.0, 1.0], [0.0, 1.0]]))
new_fieldset!(el1, "temperature thermal conductivity")
add_field!(el1, "temperature thermal conductivity", Field(0.0, 6.0))

el2 = Seg2([1, 2])
new_fieldset!(el2, "geometry")
add_field!(el2, "geometry", Field(0.0, Vector[[0.0, 0.0], [1.0, 0.0]]))

# Boundary load, linear ramp 0 -> 600 at time 0 -> 1
new_fieldset!(el2, "temperature flux")
add_field!(el2, "temperature flux", Field(0.0, 0.0))
add_field!(el2, "temperature flux", Field(1.0, 600.0))

problem1 = PlaneHeatProblem()
add_element!(problem1, el1)
add_element!(problem1, el2)

# Define Problem 2:
# - Dirichlet boundary Γ₂={0<=x<=1, y=1}, u=0 on Γ₂

el3 = Seg2([3, 4])
new_fieldset!(el3, "geometry")
add_field!(el3, "geometry", Field(0.0, Vector[[1.0, 1.0], [0.0, 1.0]]))

problem2 = DirichletProblem()
add_element!(problem2, el3)

# Create a solver for a set of problems
solver = SimpleSolver()
add_problem!(solver, problem1)
add_problem!(solver, problem2)

# Solve problem at time t=1.0 and update fields
call(solver, 1.0)

# Postprocess.
# Interpolate temperature field along boundary of Γ₁ at time t=1.0
xi = linspace([-1.0], [1.0], 5)
X = interpolate(el2, "geometry", xi, 1.0)
T = interpolate(el2, "temperature", xi, 1.0)
println(X)
println(T)

09-Oct 23:34:17:DEBUG:root:total dofs: 4


h = [0.5,0.5]
f = 600.0
Array(Float64,(8,8)) 8x8 Array{Float64,2}:
  4.0  -1.0  -2.0       -1.0       0.0  0.0  0.0       0.0     
 -1.0   4.0  -1.0       -2.0       0.0  0.0  0.0       0.0     
 -2.0  -1.0   4.0       -1.0       0.0  0.0  0.333333  0.166667
 -1.0  -2.0  -1.0        4.0       0.0  0.0  0.166667  0.333333
  0.0   0.0   0.0        0.0       0.0  0.0  0.0       0.0     
  0.0   0.0   0.0        0.0       0.0  0.0  0.0       0.0     
  0.0   0.0   0.333333   0.166667  0.0  0.0  0.0       0.0     
  0.0   0.0   0.166667   0.333333  0.0  0.0  0.0       0.0     
Array(Float64,(8,1)) 8x1 Array{Float64,2}:
 150.0
 150.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
Residual norm: 4.9227844771419234e-14
Array{T,1}[

## Comparison of results

We should have T=100 in edge.

In [3]:
@fact abs(T-100) --> 1.0e-9

Failure :: (line:-1) :: fact was false
  Expression: abs(T - 100) --> 1.0e-9
    Expected: 1.0e-9
    Occurred: [49.999999999999986,49.999999999999986,49.999999999999986,49.999999999999986,49.999999999999986]